In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
latent_dim = 256

def list_splitter(list_to_split, ratio):
    elements = len(list_to_split)
    middle = int(elements * ratio)
    return [list_to_split[:middle], list_to_split[middle:]]

In [3]:
text_en = []
text_ukr = []
chars_en = set()
chars_ukr = set()
with open("ukr.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    print(lines[:10])
for line in list_splitter(lines, 0.1)[0]:
    input_text, target_text, _ = line.lower().split("\t")
    target_text = "\t" + target_text + "\n"
    text_en.append(input_text)
    text_ukr.append(target_text)
    chars_en = chars_en.union(set(input_text))
    chars_ukr = chars_ukr.union(set(target_text))

['Go.\tЙди.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #6584257 (deniko)', 'Hi.\tВітаю!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #414700 (deniko)', 'Hi.\tПривіт.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3841503 (rmdas)', 'Hi.\tПривіт!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #5794285 (deniko)', 'Run!\tБіжіть!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5741914 (deniko)', 'Run!\tТікайте!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5741915 (deniko)', 'Run!\tБіжи!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #6452218 (deniko)', 'Wow!\tОце так!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #473081 (uaspeaker)', 'Wow!\tКлас!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2820147 (deniko)', 'Wow!\tКласно!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2820148 (deniko)']


In [4]:
for en, ukr in list(zip(text_en, text_ukr))[:5]:
    print(en, ukr)

go. 	йди.

hi. 	вітаю!

hi. 	привіт.

hi. 	привіт!

run! 	біжіть!



In [5]:
# Sort English and Ukrainian characters and convert them to lists
chars_en = sorted(list(chars_en))
chars_ukr = sorted(list(chars_ukr))

# Get the number of unique tokens for encoder and decoder
num_encoder_tokens = len(chars_en)
num_decoder_tokens = len(chars_ukr)

# Find the maximum sequence length for encoder and decoder
max_encoder_seq_length = max([len(txt) for txt in text_en])
max_decoder_seq_length = max([len(txt) for txt in text_ukr])

# Create a dictionary mapping each character to its index for encoder and decoder
input_token_index = {ch: i for i, ch in enumerate(chars_en)}
target_token_index = {ch: i for i, ch in enumerate(chars_ukr)}

# Initialize numpy arrays for encoder input, decoder input, and target data
encoder_input_data = np.zeros((len(text_en), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

# Loop through each pair of English and Ukrainian texts
for i, (en, ukr) in enumerate(zip(text_en, text_ukr)):
    # Process English text
    for t, ch in enumerate(en):
        # One-hot encode the character in the encoder input data
        encoder_input_data[i, t, input_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0

    # Process Ukrainian text
    for t, ch in enumerate(ukr):
        # One-hot encode the character in the decoder input data
        decoder_input_data[i, t, target_token_index[ch]] = 1.0
        # Set target data for all characters except the first one
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [6]:
# Define the encoder input layer with shape (None, num_encoder_tokens)
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# Create an LSTM layer for the encoder that returns the hidden state and cell state
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Store the encoder states (hidden state and cell state)
encoder_states = [state_h, state_c]

# Define the decoder input layer with shape (None, num_decoder_tokens)
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# Create an LSTM layer for the decoder that returns sequences, hidden state, and cell state
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Apply a Dense layer to produce decoder outputs with softmax activation
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the overall model with encoder and decoder inputs, and decoder outputs
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [7]:
# Compile the model with specified optimizer, loss function, and metrics
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model using the provided input and target data
model.fit(
    [encoder_input_data, decoder_input_data],  # Input data for the encoder and decoder
    decoder_target_data,  # Target data for the decoder
    batch_size=64,  # Number of samples per gradient update
    epochs=100,  # Number of epochs (iterations over the entire dataset)
    validation_split=0.2,  # Fraction of training data to be used as validation data
)

model.save("s2s")

Epoch 1/100
199/199 [==============================] - 48s 234ms/step - loss: 1.1226 - accuracy: 0.7411 - val_loss: 1.0097 - val_accuracy: 0.7353
Epoch 2/100
199/199 [==============================] - 50s 253ms/step - loss: 0.8090 - accuracy: 0.7858 - val_loss: 0.8441 - val_accuracy: 0.7679
Epoch 3/100
199/199 [==============================] - 61s 307ms/step - loss: 0.6725 - accuracy: 0.8092 - val_loss: 0.7631 - val_accuracy: 0.7798
Epoch 4/100
199/199 [==============================] - 58s 292ms/step - loss: 0.6180 - accuracy: 0.8209 - val_loss: 0.7263 - val_accuracy: 0.7890
Epoch 5/100
199/199 [==============================] - 54s 273ms/step - loss: 0.5840 - accuracy: 0.8300 - val_loss: 0.6907 - val_accuracy: 0.7993
Epoch 6/100
199/199 [==============================] - 63s 317ms/step - loss: 0.5587 - accuracy: 0.8365 - val_loss: 0.6736 - val_accuracy: 0.8033
Epoch 7/100
199/199 [==============================] - 57s 285ms/step - loss: 0.5382 - accuracy: 0.8412 - val_loss: 0.6558 -

INFO:tensorflow:Assets written to: s2s/assets


In [11]:
# Load the sequence-to-sequence (s2s) model from a saved file
model = keras.models.load_model("s2s")

# Extract the encoder part of the model to create a separate encoder model
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

# Extract the decoder part of the model to create a separate decoder model
decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Create dictionaries to map indices back to characters for input and target sequences
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [12]:
def decode_sequence(input_seq):
    # Get the initial states from the encoder model
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with a start token "\t"
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Initialize variables for loop termination and decoded sentence
    stop_condition = False
    decoded_sentence = ""

    # Decode sequence in a loop until a newline or maximum sequence length is reached
    while not stop_condition:
        # Predict the next token in the sequence using the decoder model
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the index of the most probable token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Convert the index to the corresponding character
        sampled_char = reverse_target_char_index[sampled_token_index]

        # Append the character to the decoded sentence
        decoded_sentence += sampled_char

        # Check for stopping conditions (newline or maximum length)
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update the states for the next iteration
        states_value = [h, c]

    # Return the final decoded sentence
    return decoded_sentence


In [13]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", text_en[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 15ms/step
-
Input sentence: go.
Decoded sentence: йди.

1/1 [==============================] - 0s 14ms/step
-
Input sentence: hi.
Decoded sentence: привіт!

1/1 [==============================] - 0s 16ms/step
-
Input sentence: hi.
Decoded sentence: привіт!

1/1 [==============================] - 0s 17ms/step
-
Input sentence: hi.
Decoded sentence: привіт!

1/1 [==============================] - 0s 14ms/step
-
Input sentence: run!
Decoded sentence: біжи!

1/1 [==============================] - 0s 14ms/step
-
Input sentence: run!
Decoded sentence: біжи!

1/1 [==============================] - 0s 15ms/step
-
Input sentence: run!
Decoded sentence: біжи!

1/1 [==============================] - 0s 16ms/step
-
Input sentence: wow!
Decoded sentence: оце так!

1/1 [==============================] - 0s 16ms/step
-
Input sentence: wow!
Decoded sentence: оце так!

1/1 [==============================] - 0s 17ms/step
-
Input sentence: wow!
Decoded sentence: 